Name: Praneeth Balakrishna

Matr num: 6868873

Collaborator: Pramod Mahajan 
 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import json

# Download the dataset

In [2]:
def get_mnist():
    # The code to download the mnist data original came from
    # https://cntk.ai/pythondocs/CNTK_103A_MNIST_DataLoader.html
    
    import gzip
    import numpy as np
    import os
    import struct

    try: 
        from urllib.request import urlretrieve 
    except ImportError: 
        from urllib import urlretrieve

    def load_data(src, num_samples):
        print("Downloading " + src)
        gzfname, h = urlretrieve(src, "./delete.me")
        print("Done.")
        try:
            with gzip.open(gzfname) as gz:
                n = struct.unpack("I", gz.read(4))
                # Read magic number.
                if n[0] != 0x3080000:
                    raise Exception("Invalid file: unexpected magic number.")
                # Read number of entries.
                n = struct.unpack(">I", gz.read(4))[0]
                if n != num_samples:
                    raise Exception(
                        "Invalid file: expected {0} entries.".format(num_samples)
                    )
                crow = struct.unpack(">I", gz.read(4))[0]
                ccol = struct.unpack(">I", gz.read(4))[0]
                if crow != 28 or ccol != 28:
                    raise Exception(
                        "Invalid file: expected 28 rows/cols per image."
                    )
                # Read data.
                res = np.frombuffer(
                    gz.read(num_samples * crow * ccol), dtype=np.uint8
                )
        finally:
            os.remove(gzfname)
        return res.reshape((num_samples, crow, ccol)) / 256


    def load_labels(src, num_samples):
        print("Downloading " + src)
        gzfname, h = urlretrieve(src, "./delete.me")
        print("Done.")
        try:
            with gzip.open(gzfname) as gz:
                n = struct.unpack("I", gz.read(4))
                # Read magic number.
                if n[0] != 0x1080000:
                    raise Exception("Invalid file: unexpected magic number.")
                # Read number of entries.
                n = struct.unpack(">I", gz.read(4))
                if n[0] != num_samples:
                    raise Exception(
                        "Invalid file: expected {0} rows.".format(num_samples)
                    )
                # Read labels.
                res = np.frombuffer(gz.read(num_samples), dtype=np.uint8)
        finally:
            os.remove(gzfname)
        return res.reshape((num_samples))


    def try_download(data_source, label_source, num_samples):
        data = load_data(data_source, num_samples)
        labels = load_labels(label_source, num_samples)
        return data, labels
    
    
    # Not sure why, but yann lecun's website does no longer support 
    # simple downloader. (e.g. urlretrieve and wget fail, while curl work)
    # Since not everyone has linux, use a mirror from uni server.
    #     server = 'http://yann.lecun.com/exdb/mnist'
    server = 'https://raw.githubusercontent.com/fgnt/mnist/master'
    
    # URLs for the train image and label data
    url_train_image = f'{server}/train-images-idx3-ubyte.gz'
    url_train_labels = f'{server}/train-labels-idx1-ubyte.gz'
    num_train_samples = 60000

    print("Downloading train data")
    train_features, train_labels = try_download(url_train_image, url_train_labels, num_train_samples)

    # URLs for the test image and label data
    url_test_image = f'{server}/t10k-images-idx3-ubyte.gz'
    url_test_labels = f'{server}/t10k-labels-idx1-ubyte.gz'
    num_test_samples = 10000

    print("Downloading test data")
    test_features, test_labels = try_download(url_test_image, url_test_labels, num_test_samples)
    
    return train_features, train_labels, test_features, test_labels

In [3]:
train_features, train_labels, test_features, test_labels = get_mnist()

Done.
Done.
Done.
Done.


In [4]:
#Reshape the data
train_features_reshape = train_features.reshape(-1, 28*28)
test_features_reshape = test_features.reshape(-1, 28*28)


In [5]:
class Variable:
    def __init__(self, value, operation):
        self.value = np.array(value)
        self.operation = operation
        
    
#     def backprop(self):
#         # We define the backpropagation code later.
#         return backprop(self)
    
class Parameter(Variable):
    """
    This class should be used for Variables that are learnable.
    You can later use this class to distinguish learnable variables
    from other variables (`isinstance(variable, Parameter)`).
    """
    def __init__(self, value):
        super().__init__(value, operation=None)
        self.gradient = np.zeros_like(self.value)
        
class Input(Variable):
    """
    This class should be used as wrapper for inputs that are not learnable.
    """
    def __init__(self, value):
        super().__init__(value, operation=None)
        

class Layer:
    """ Abstract layer class.  
    """
    def __init__(self):
        self.parameters = []
    
    def forward(self, X):
        """ to be implemented in the subclasses
            implement the forward transformation in each layer
        """
        raise NotImplementedError
        

        
    def add_param(self, values):
        """ All trainable parameters of the layer are appended to the parameters attribute
        """
        param = Parameter(values)
        self.parameters.append(param)
        return param
    
    def update_parameters(self, optimizer):
        """ called after each backpropogation
        param optimizer: here it is the SGD optimizer
        """
        for param in self.parameters:
            optimizer.update(param)
        

class AffineLayer(Layer):
    """ Layer class that performs a linear transformation on the data in a MLP
    """
    def __init__(self, in_units, out_units):
        """ Initialize weights and biases here. Dimensions based on in_units and out_units
        """
        super().__init__()
        small_value = 0.01
        weight_vals = np.random.uniform(
                    size=[in_units, out_units],
                    low=-small_value,
                    high=small_value
                    )
        self.W = self.add_param(weight_vals)
        self.b = self.add_param(np.zeros(shape=out_units))
        
    def forward(self, X):
        def backward(D):
            #print(self.__class__.__name__)
            
            self.W.gradient = self.W.gradient + X.T @ D  
                                  ## todo to check for correctness (is += required)
            self.b.gradient = self.b.gradient + np.sum(D,axis=0) 
           
            return D @ self.W.value.T
       
       
        return X @ self.W.value + self.b.value, backward
    
    
class Sequential(Layer):
    """ Models the multi layer perceptron. layers can only be cascaded one after the other.
    """
    
    def __init__(self, *layers):
        """  args *layers: pass multiple cascaded layers of the MLP as arguments to this class constructor
        """
        super().__init__()
        self.layers = layers
        for layer in layers:
            self.parameters.extend(layer.parameters) 
        
    def forward(self, X):
        backprops = []
        op = X
        for layer in self.layers:
            op, backprop = layer.forward(op)
            backprops.append(backprop)

        def backward(D):
            for backprop in reversed(backprops):
                D = backprop(D)
            return D
        return op , backward

In [6]:
class ReLU(Layer):
    def forward(self, X):
        mask = X > 0
        return X * mask, lambda D: D * mask
        
    
    
class Sigmoid(Layer):
    def forward(self, X):
        op = 1/(1+np.exp(-X))
        
        def backward(D):
            """ The gradient at output is computed with respect to output of the last affine layer
            hence the backward just propogates the previous derivative.
            """
            return D #* op * (1 - op)
        
        return op, backward
    
class SGDOptimizer():
    def __init__(self, lr=0.1):
        self.lr = lr

    def update(self, param):
        param.value = param.value - self.lr * param.gradient 
        param.gradient.fill(0)

In [7]:
def cross_entropy(predictions, targets, epsilon=1e-11):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions ndarray
           targets  ndarray        
    Returns: scalar Cross Entropy loss
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions))/N
    return ce , predictions - targets   #todo verify derivative of ce loss

    

def one_hot_encoder(x_label):
    """ returns one hot encoded arrays for the input data labels
    """
    rows = x_label.shape[0]
    oh_x = np.zeros((rows, 10))
    for i in range(rows):
        oh_x[i][x_label[i]] = 1
    
    return oh_x



In [8]:


class DigitLearner():
    """ Class to train the MLP. 
    """
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss = loss_fn
        self.optimizer = optimizer
        
    def fit_batch(self,X,Y):
        """ Performs forward and backward operation on one batch of training samples
            param X : A batch of training data
            param Y : A corresponding batch of training labels
        """
        Y_, backward = self.model.forward(X)
        L , D = self.loss(Y_, Y)          
                                
        backward(D)
        self.model.update_parameters(self.optimizer)
        return L
  
    def fit(self, X, Y, epochs, bs):
        """ Fits the entire training data in batches
            param X : entire training data
            param Y : entire training labels
            param bs : batch size for SGD
            return : an array of training losses of each epoch 
        """
        losses = []
        
     
        for epoch in range(epochs):
            p = np.random.permutation(len(X))
            L = 0
           
            for i in range(0, len(X), bs):
                X_batch = X[p[i:i + bs]]
                Y_batch = Y[p[i:i + bs]]
                L += self.fit_batch(X_batch, Y_batch)
                  
            losses.append(L)

        
        print("Training complete")
        return losses
    
    def predict(self, xtest):
        """ predict the output given an input batch of samples
        """
        ypred, _ = self.model.forward(xtest)
        return ypred.argmax(axis=-1)
    
    def accuracy(y_pred, y_test):
        return np.sum(y_pred == y_test)/y_pred.shape[0]
    
    def dump_params(self,filename):
        """ writes the trained weights to a json file
        """
        with open(filename, 'w') as fp:
            json.dump([p.value.tolist() for p in self.model.parameters],fp)
            print("weights written to file")
            
    def load_params(self, filename):
        """ loads weights from a json file and initializes the ANN with the loaded weights
        """
        with open(filename) as fp:
            loaded_params = json.load(fp)
        for p, p_value in zip(self.model.parameters, loaded_params):
            p.value = p_value
        print("Loaded given weights!!")
            
        
    

In [9]:
#create a new empty model here to load weights

hidden_neurons = 512

# create an ANN model with the same architecture as the earlier trained model.
mt_model = DigitLearner(
    Sequential(
        AffineLayer(784, hidden_neurons), 
        ReLU(), 
        AffineLayer(hidden_neurons, 10),
        Sigmoid()        
    ), 
    cross_entropy, 
    SGDOptimizer()
)



In [10]:
# load weights from a saved json file in a new model
mt_model.load_params("1lyr_512_0.008_99_17_ep_wt.json")

Loaded given weights!!


In [11]:
# use the new empty model with weights loaded from json to make prediction
tacc = DigitLearner.accuracy(mt_model.predict(test_features_reshape), test_labels)
print("The prediction accuracy on test dataset is:", tacc*100,"%")

The prediction accuracy on test dataset is: 99.17 %
